In [8]:
import pickle
import pandas as pd
import numpy as np
import torch
import os
import data

from utils import get_topic_coherence as get_tc
from utils import get_document_frequency
import random

In [12]:
random.randint(1,200)

112

In [2]:
import torch
import os
import data


def child():
    import torch
    torch.set_num_threads(1)


import mkl
from torch import multiprocessing as mp

os.environ['MKL_THREADING_LAYER'] = 'GNU'
mp.set_start_method('spawn')
p = mp.Process(target=child)
p.start()
p.join()

# Если вдруг нужны тета

In [3]:
device = 'cpu'
def get_batch(tokens, counts, ind, vocab_size, device, emsize=300):
    batch_size = len(ind)
    data_batch = np.zeros((batch_size, vocab_size))
    for i, doc_id in enumerate(ind):
        doc = tokens[doc_id]
        count = counts[doc_id]
        L = count.shape[1]
        if len(doc) == 1: 
            doc = [doc.squeeze()]
            count = [count.squeeze()]
        else:
            doc = doc.squeeze()
            count = count.squeeze()
        if doc_id != -1:
            for j, word in enumerate(doc):
                data_batch[i, word] = count[j]
    return data_batch



def get_full_theta(indices, tokens, counts, vocab_size, bow_norm=0):
    all_theta = []
    for idx, ind in enumerate(indices):
        data_batch = data.get_batch(tokens, counts, ind, vocab_size, 'cpu')
        sums = data_batch.sum(1).unsqueeze(1)
        if bow_norm:
            normalized_data_batch = data_batch / sums
        else:
            normalized_data_batch = data_batch
        theta, _ = model.get_theta(normalized_data_batch)
        theta = theta.detach().numpy()
    all_theta.append(theta) ##сэмплированные темы
    return all_theta

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

# Цикл обучения

In [ ]:
# читаем данные из даатсета лента
#vocab, train, valid, test = data.get_data(os.path.join('/home/skoltsov/koltcov_sergei/data/lenta'))


# читаем данные по датасету 20news
vocab, train, valid, test = data.get_data(os.path.join('/home/skoltsov/koltcov_sergei/data/20news'))

#vocab, train, valid, test = data.get_data(os.path.join('/home/skoltsov/koltcov_sergei/data/WOS_short'))

#vocab, train, valid, test = data.get_data(os.path.join('/home/skoltsov/koltcov_sergei/data/20news_clear'))



train_tokens = train['tokens']
train_counts = train['counts']
    
# чтение эмбедингов датасета лента
#path_emb = '/home/skoltsov/koltcov_sergei/embeddings/'
#results = '/home/skoltsov/koltcov_sergei/results/lenta/' 

# чтение эмбедингов датасета 20topicsnews
path_emb = '/home/skoltsov/koltcov_sergei/embeddings/'
results = '/home/skoltsov/koltcov_sergei/results/20topicsnews/' 


#results = '/home/skoltsov/koltcov_sergei/results/WOS_short/'
#results = '/home/skoltsov/koltcov_sergei/results/WOS_short/'
#results = '/home/skoltsov/koltcov_sergei/results/20news_clear/'
    
for i in os.listdir(path_emb):
    emb = i[:-4]
    emb_path = 'embeddings/' + i
    
    print(emb_path)
    # читаем эмбединги
    with open(emb_path, 'rb') as f:
        for l in f:
            line = l.decode('utf8').split()
            #line = l.decode('cp1251').split()
            emb_size = len(line)-1
            break
    for train in [0,1]:
        all_Renyi = []
        coherence = []
        for topics in range(2,50):
            print('Current topic: ', topics)
            # генерация нового сида
            myseed = random.randint(1,200)
            

# лента
#            !python main.py --dataset lenta_lem_1 --data_path data/lenta --emb_path {emb_path} --save_path results/lenta/ --batch_size 1000 --num_topics {topics} --rho_size {emb_size} --emb_size {emb_size} --t_hidden_size 300 --theta_act selu --train_embeddings {train} --lr 0.01 --optimizer adam --epochs 200 --seed 200 --clip 0.1 --encoding cp1251
# 20topicsnews
            !python main.py --dataset 20topicsnews_lem_1 --data_path data/20news --emb_path {emb_path} --save_path results/20topicsnews/ --batch_size 1000 --num_topics {topics} --rho_size {emb_size} --emb_size {emb_size} --t_hidden_size 300 --theta_act selu --train_embeddings {train} --lr 0.01 --optimizer adam --epochs 200 --seed 200 --clip 0.1 --encoding utf8
         

embeddings/wiki-news-300d-1M.txt
Current topic:  2


=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
Training an Embedded Topic Model on 20TOPICSNEWS_LEM_1 with the following settings: Namespace(anneal_lr=0, batch_size=1000, bow_norm=1, clip=0.1, data_path='data/20news', dataset='20topicsnews_lem_1', emb_path='embeddings/wiki-news-300d-1M.txt', emb_size=300, embeddings_dim=torch.Size([12110, 300]), enc_drop=0.0, encoding='utf8', epochs=200, eval_batch_size=1000, load_from='', log_interval=2, lr=0.01, lr_factor=4.0, mode='train', nonmono=10, num_docs_test=499, num_docs_test_1=499, num_docs_test_2=499, num_docs_train=14766, num_docs_valid=100, num_topics=2, num_words=10, optimizer='adam', rho_size=300, save_path='results/20topicsnews/', seed=200, t_hidden_size=300, tc=0, td=0, theta_act='selu', train_embeddings=0, visualize_every=10, voc

Epoch: 6 .. batch: 2/15 .. LR: 0.01 .. KL_theta: 0.96 .. Rec_loss: 1114.87 .. NELBO: 1115.83
Epoch: 6 .. batch: 4/15 .. LR: 0.01 .. KL_theta: 1.06 .. Rec_loss: 1082.72 .. NELBO: 1083.78
Epoch: 6 .. batch: 6/15 .. LR: 0.01 .. KL_theta: 1.09 .. Rec_loss: 1052.42 .. NELBO: 1053.51
Epoch: 6 .. batch: 8/15 .. LR: 0.01 .. KL_theta: 1.04 .. Rec_loss: 1050.79 .. NELBO: 1051.83
Epoch: 6 .. batch: 10/15 .. LR: 0.01 .. KL_theta: 1.08 .. Rec_loss: 1039.79 .. NELBO: 1040.87
Epoch: 6 .. batch: 12/15 .. LR: 0.01 .. KL_theta: 1.13 .. Rec_loss: 1037.83 .. NELBO: 1038.96
Epoch: 6 .. batch: 14/15 .. LR: 0.01 .. KL_theta: 1.11 .. Rec_loss: 1037.75 .. NELBO: 1038.86
****************************************************************************************************
Epoch----->6 .. LR: 0.01 .. KL_theta: 1.11 .. Rec_loss: 1037.75 .. NELBO: 1038.86
****************************************************************************************************
**************************************************************

****************************************************************************************************
VAL Doc Completion PPL: 6950.3
****************************************************************************************************
Epoch: 13 .. batch: 2/15 .. LR: 0.01 .. KL_theta: 1.13 .. Rec_loss: 1065.98 .. NELBO: 1067.11
Epoch: 13 .. batch: 4/15 .. LR: 0.01 .. KL_theta: 1.12 .. Rec_loss: 1048.35 .. NELBO: 1049.47
Epoch: 13 .. batch: 6/15 .. LR: 0.01 .. KL_theta: 1.22 .. Rec_loss: 1031.64 .. NELBO: 1032.86
Epoch: 13 .. batch: 8/15 .. LR: 0.01 .. KL_theta: 1.19 .. Rec_loss: 1029.15 .. NELBO: 1030.34
Epoch: 13 .. batch: 10/15 .. LR: 0.01 .. KL_theta: 1.16 .. Rec_loss: 1040.47 .. NELBO: 1041.63
Epoch: 13 .. batch: 12/15 .. LR: 0.01 .. KL_theta: 1.18 .. Rec_loss: 1032.43 .. NELBO: 1033.61
Epoch: 13 .. batch: 14/15 .. LR: 0.01 .. KL_theta: 1.17 .. Rec_loss: 1037.07 .. NELBO: 1038.24
****************************************************************************************************
Epoch

****************************************************************************************************
VAL Doc Completion PPL: 6952.8
****************************************************************************************************
Epoch: 20 .. batch: 2/15 .. LR: 0.01 .. KL_theta: 1.25 .. Rec_loss: 966.14 .. NELBO: 967.39
Epoch: 20 .. batch: 4/15 .. LR: 0.01 .. KL_theta: 1.16 .. Rec_loss: 997.47 .. NELBO: 998.63
Epoch: 20 .. batch: 6/15 .. LR: 0.01 .. KL_theta: 1.16 .. Rec_loss: 1026.93 .. NELBO: 1028.09
Epoch: 20 .. batch: 8/15 .. LR: 0.01 .. KL_theta: 1.2 .. Rec_loss: 1025.89 .. NELBO: 1027.09
Epoch: 20 .. batch: 10/15 .. LR: 0.01 .. KL_theta: 1.18 .. Rec_loss: 1031.76 .. NELBO: 1032.94
Epoch: 20 .. batch: 12/15 .. LR: 0.01 .. KL_theta: 1.15 .. Rec_loss: 1033.56 .. NELBO: 1034.71
Epoch: 20 .. batch: 14/15 .. LR: 0.01 .. KL_theta: 1.16 .. Rec_loss: 1035.2 .. NELBO: 1036.36
****************************************************************************************************
Epoch----->

****************************************************************************************************
VAL Doc Completion PPL: 6965.2
****************************************************************************************************
Epoch: 27 .. batch: 2/15 .. LR: 0.01 .. KL_theta: 1.05 .. Rec_loss: 984.89 .. NELBO: 985.94
Epoch: 27 .. batch: 4/15 .. LR: 0.01 .. KL_theta: 1.09 .. Rec_loss: 1005.28 .. NELBO: 1006.37
Epoch: 27 .. batch: 6/15 .. LR: 0.01 .. KL_theta: 1.11 .. Rec_loss: 1019.38 .. NELBO: 1020.49
Epoch: 27 .. batch: 8/15 .. LR: 0.01 .. KL_theta: 1.12 .. Rec_loss: 1040.54 .. NELBO: 1041.66
Epoch: 27 .. batch: 10/15 .. LR: 0.01 .. KL_theta: 1.13 .. Rec_loss: 1036.46 .. NELBO: 1037.59
Epoch: 27 .. batch: 12/15 .. LR: 0.01 .. KL_theta: 1.14 .. Rec_loss: 1038.37 .. NELBO: 1039.51
Epoch: 27 .. batch: 14/15 .. LR: 0.01 .. KL_theta: 1.13 .. Rec_loss: 1036.63 .. NELBO: 1037.76
****************************************************************************************************
Epoch--

****************************************************************************************************
VAL Doc Completion PPL: 6967.3
****************************************************************************************************
Epoch: 34 .. batch: 2/15 .. LR: 0.01 .. KL_theta: 1.17 .. Rec_loss: 1038.14 .. NELBO: 1039.31
Epoch: 34 .. batch: 4/15 .. LR: 0.01 .. KL_theta: 1.17 .. Rec_loss: 1043.55 .. NELBO: 1044.72
Epoch: 34 .. batch: 6/15 .. LR: 0.01 .. KL_theta: 1.14 .. Rec_loss: 1029.7 .. NELBO: 1030.84
Epoch: 34 .. batch: 8/15 .. LR: 0.01 .. KL_theta: 1.14 .. Rec_loss: 1031.97 .. NELBO: 1033.11
Epoch: 34 .. batch: 10/15 .. LR: 0.01 .. KL_theta: 1.17 .. Rec_loss: 1038.7 .. NELBO: 1039.87
Epoch: 34 .. batch: 12/15 .. LR: 0.01 .. KL_theta: 1.16 .. Rec_loss: 1046.19 .. NELBO: 1047.35
Epoch: 34 .. batch: 14/15 .. LR: 0.01 .. KL_theta: 1.15 .. Rec_loss: 1034.01 .. NELBO: 1035.16
****************************************************************************************************
Epoch--

****************************************************************************************************
VAL Doc Completion PPL: 6924.7
****************************************************************************************************
####################################################################################################
Visualize topics...
Topic 0: ['have', 'experimenter', 'rejoice', 'betz', 'hadn', 'badge', 'synoptic', 'hear', 'rigel']
Topic 1: ['they', 'experimenter', 'datum', 'queiser', 'rigel', 'synoptic', 'hartill', 'smartdrv', 'thank']
Epoch: 41 .. batch: 2/15 .. LR: 0.01 .. KL_theta: 1.07 .. Rec_loss: 1056.27 .. NELBO: 1057.34
Epoch: 41 .. batch: 4/15 .. LR: 0.01 .. KL_theta: 1.13 .. Rec_loss: 1041.85 .. NELBO: 1042.98
Epoch: 41 .. batch: 6/15 .. LR: 0.01 .. KL_theta: 1.18 .. Rec_loss: 1058.91 .. NELBO: 1060.09
Epoch: 41 .. batch: 8/15 .. LR: 0.01 .. KL_theta: 1.15 .. Rec_loss: 1051.46 .. NELBO: 1052.61
Epoch: 41 .. batch: 10/15 .. LR: 0.01 .. KL_theta: 1.18 .. Rec_l

****************************************************************************************************
VAL Doc Completion PPL: 6957.8
****************************************************************************************************
Epoch: 48 .. batch: 2/15 .. LR: 0.01 .. KL_theta: 1.18 .. Rec_loss: 1030.08 .. NELBO: 1031.26
Epoch: 48 .. batch: 4/15 .. LR: 0.01 .. KL_theta: 1.17 .. Rec_loss: 1015.44 .. NELBO: 1016.61
Epoch: 48 .. batch: 6/15 .. LR: 0.01 .. KL_theta: 1.15 .. Rec_loss: 1015.05 .. NELBO: 1016.2
